In [ ]:
!pip install flask flask-ngrok librosa torch transformers numpy pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor
import librosa
import torch
import numpy as np
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

 # Load the model and feature extractor (same as before)
model_id = "firdhokk/speech-emotion-recognition-with-openai-whisper-large-v3"
model = AutoModelForAudioClassification.from_pretrained(model_id)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, do_normalize=True)
id2label = model.config.id2label

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.55G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [ ]:
from flask import Flask
from pyngrok import ngrok
import os

app = Flask(__name__)

def preprocess_audio(audio_path, max_duration=30.0):
    """Load and preprocess audio for the model."""
    try:
        audio_array, sampling_rate = librosa.load(audio_path, sr=feature_extractor.sampling_rate)

        # Ensure max duration (trim or pad audio)
        max_length = int(feature_extractor.sampling_rate * max_duration)
        if len(audio_array) > max_length:
            audio_array = audio_array[:max_length]
        else:
            audio_array = np.pad(audio_array, (0, max_length - len(audio_array)))

        # Extract features
        inputs = feature_extractor(
            audio_array,
            sampling_rate=feature_extractor.sampling_rate,
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )
        return inputs
    except Exception as e:
        raise RuntimeError(f"Audio preprocessing error: {str(e)}")

def predict_emotion(audio_path):
    """Predict emotion from an audio file."""
    try:
        inputs = preprocess_audio(audio_path)

        # Move model and inputs to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        # Get predicted label
        logits = outputs.logits
        predicted_id = torch.argmax(logits, dim=-1).item()
        predicted_label = id2label[predicted_id]

        return predicted_label
    except Exception as e:
        raise RuntimeError(f"Prediction error: {str(e)}")

@app.route('/predict', methods=['POST'])
def predict():
    print("📥 Request received at /predict")
    if 'file' not in request.files:
        print("❌ No file provided")
        return jsonify({"error": "No file provided"}), 400

    file = request.files['file']
    file_path = os.path.join("/tmp", "temp_audio.wav")
    file.save(file_path)
    print(f"✅ File saved: {file_path}")

    try:
        print("🎭 Running emotion prediction...")
        emotion = predict_emotion(file_path)
        print(f"🎯 Prediction: {emotion}")
        return jsonify({"emotion": emotion})
    except Exception as e:
        print(f"🚨 Prediction Error: {str(e)}")
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
  ngrok.set_auth_token("2uEUlCDUG1fCMwiC4ps14UMil2d_471PqehH5fokBrxsnH9qP")
  ngrok_tunnel = ngrok.connect(5000)
  print('Public URL:', ngrok_tunnel.public_url)
  app.run()

Public URL: https://e54a-104-199-173-178.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:19:57] "POST /predict HTTP/1.1" 200 -


🎯 Prediction: surprised
